<a href="https://colab.research.google.com/github/mcstllns/DeepLearning/blob/main/P04_MLP_regresion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Perceptron Multicapa para una regresión lineal

En este ejercicio vamos a crear una red para pronosticar el consumo de gasolina de los coches. Los datos provienen de la universidad de California y son ampliamente utilizados en los curos de Deep Learning.

La información de los datos la tienes en este enlace, y son un conjunto de datos que contiene las siguientes variables:

1. mpg: continuous
2. cylinders: multi-valued discrete
3. displacement: continuous
4. horsepower: continuous
5. weight: continuous
6. acceleration: continuous
7. model year: multi-valued discrete
8. origin: multi-valued discrete
9. car name: string (unique for each instance)

El objetivo es pronosticar mpg que es el consumo de gasolina de los coches (millas por galón) basándose en las otras variables.

La variable origin esta compuesta por tres valores:

- 1: USA
- 2: Europe
- 3: Japan


Así que lo mejor es hacer un one hot con ella.

Como la variable MPG es continua vamos a utilizar un capa final lineal y como función de pérdida el mse.

Además vamos a separar el conjunto de datos en tres: train, dev y test como se ha visto en la teoría y haremos unos gráficos para ver la evolución del aprendizaje y el ajuste final.



In [ ]:
# paquetes y librerías que vamos a necesitar

import tensorflow as tf

print(tf.__version__)

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import time


In [ ]:
# funciones que nos van a ayudar

# normaliza un conjunto de datos
def norm(x, st):
    return((x - st['mean'])/st['std'])

# un plot de la historia del ajuste
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error')
  plt.plot(hist['epoch'], hist['mse'],'r--',
           label='Training Error')
  plt.plot(hist['epoch'], hist['val_mse'],'b',
           label = 'Validation Error')
  plt.ylim([0,20])
  plt.legend()
  plt.show() # un €



In [ ]:
# Cargamos los datos
# Keras nos permite cargar directamente un fichero que este en internet 

# Especificamos la direccion donde esta el fichero y su nombre y como lo queremos guardar
dataset_path = tf.keras.utils.get_file("auto-mpg.data",
                                       "https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")

# El csv viene sin ninguna cabezera estos son los nombres de las variables
column_names = ['MPG', 'Cylinders','Displacement','Horsepower','Weight','Acceleration','Model year','Origin']

# se lee el fichero
data = pd.read_csv(dataset_path, names=column_names, na_values="?",
                          comment='\t', sep=" ", skipinitialspace=True)

In [ ]:
# vemos las primeras filas de los datos
data.head()

In [ ]:
# PREPROCESAMIENTO --------------------

# El fichero tiene datos perdidos y hay que eliminar las filas donde estan
data = data.dropna() 



# La variable origin viene codificada de forma numerica pero la pasamos a un one hot

# no te preocupes por este codigo, cosas de python
o = data.pop('Origin')
data['USA'] = (o == 1)*1.0
data['Europe'] = (o == 2)*2.0
data['Japan'] = (o == 3)*3.0

# comprobamos que ahora tenemos una variable para cada pais de origen
data.head()


In [ ]:
# Como el train y test sets no estan creados los creamos ahora aleatoriamente

x_train = data.sample(frac=0.8, random_state=0)
x_test = data.drop(x_train.index)

y_train = x_train.pop('MPG')
y_test = x_test.pop('MPG')


In [ ]:
x_train.head()

In [ ]:
y_train.head()

Si quieres investiga un poco el código anterior para aprender un poco de python o si prefieres déjalo como está. Lo que hemos hecho es:
1. x_train se crea eligiendo aleatoriamente el 80% de las dilas de data
1. x_test son simplemente las que no han sido elegidas
1. y_train e y_test son solo la variable MPG de los conjuntos de datos anteriores


In [ ]:
# como vemos las variables en x tienen valores muy dispares, vamos a normalizarlas
# para que funcione adecuadamente el optimizador y no haya problemas de
# exploding gradient

# No es necesario normalizar y porque es la que intentamos predecir y queremos ver
# cuanto nos estamos equivocando en la escala real

In [ ]:
# calculamos los estadisticos descriptivos para x_train
x_train_stats = x_train.describe().transpose()
x_train_stats.head()

In [ ]:
# calculamos los estadisticos descriptivos para x_test
x_test_stats = x_test.describe().transpose()
x_test_stats.head()

In [ ]:
# normalizamos x_train y x_test

x_train_norm = norm(x_train, x_train_stats)
x_train_norm.head()

In [ ]:
x_test_norm = norm(x_test, x_test_stats)
x_test_norm.head()

In [ ]:
# comprobamos que lo hemos hecho bien

x_train_norm.describe().transpose()

Está perfecto, las medias cercanas a cero (2.093159e-16) y las sd en 1.0

In [ ]:
# Modelo --------------------------


# Configuramos la topología

model = Sequential()
model.add(Dense(64, activation = 'relu', input_shape=[( len(x_train_norm.keys()) )]))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(1, activation = 'linear'))

model.summary()

In [ ]:
# compilamos

model.compile(loss='mse',
              optimizer = tf.keras.optimizers.RMSprop(learning_rate = 0.001),
              metrics = ['mse','mae'])

In [ ]:
# entrenamos
start = time.time()
history = model.fit(x_train_norm, y_train,
                    epochs = 300,
                    validation_split = 0.2,  # divide automaticamente el train
                                             # en train (80%) y dev (20%)
                    verbose = 0) # para evitar que se llene toda la pantalla

end = time.time()
print('Tiempo de ejecución:', end - start)

In [ ]:
# Un grafico para ver el ajuste en los dos sets

plot_history(history)

In [ ]:
# Evaluamos el modelo con el conjunto de validacion
model.evaluate(x_test_norm, y_test)

In [ ]:
# Como las variables son cuantitativas un grafico de dispersion podría estar perfecto

yp = model.predict(x_test_norm)


df = pd.DataFrame({'y': y_test, 'yp': yp.flatten()})
df.plot.scatter(x='y', y='yp')
plt.show()

# Ejercicio

Prueba a modificar los hiperparámetros de la red e incluso su arquitectura para obtener una mayor velocidad en la red. Por ejemplo podrías hacer lo siguiente:

1. Prueba a utilizar otros tipos de neuronas (RelU o tanh) y otros optimizadores para mejorar el tiempo de aprendizaje
1. Simplifica al máximo la red pero sin una pérdida significativa de predicción

--- 

Haz las pruebas que consideres interesantes e interpreta los resultados.

Pega el código empleado (solo las partes que has cambiado) en la ventana de abajo

Para ayudarte en las tareas simplemente usa google, te aportará miles de ejemplos

Recuerda que los resultados tienes que subirlos a un cuestionario de moodle

---

__Opcional__: aprovechando que tenemos en df los valores de y_test y de yp (los pronosticados) calcula el R2 de esta recta que nos indicará la calidad del pronóstico usando un lenguaje más estadístico.